In [19]:
# Import necessary libraries for data manipulation and analysis
import pandas as pd
import numpy as np

In [20]:
# Load the engineered retail dataset from a CSV file into a pandas DataFrame
# Using sep=',' to explicitly specify the comma separator, although the file might have other formatting issues.
merged_df = pd.read_csv("/content/engineered_retail_dataset__for_ml.csv", sep=',')
# Display the first 5 rows of the DataFrame to get a preview of the data
display(merged_df.head())

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Weekly_Sales_Lag2,Weekly_Sales_Lag3,Weekly_Sales_MA4,Weekly_Sales_STD4,Weekly_Sales_MA12,Weekly_Sales_STD12,IsHoliday_Lag1,IsHoliday_Lead1,Rolling_4,Rolling_12
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,0.00,0.00,0.00,0.000000,0.0,0.0,False,True,0.00,0.0
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,...,0.00,0.00,0.00,0.000000,0.0,0.0,False,False,0.00,0.0
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,...,24924.50,0.00,0.00,0.000000,0.0,0.0,True,False,0.00,0.0
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,...,46039.49,24924.50,0.00,0.000000,0.0,0.0,False,False,0.00,0.0
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,...,41595.55,46039.49,32990.77,12832.106391,0.0,0.0,False,False,32990.77,0.0


In [21]:
# Convert 'Date' column to datetime objects to enable time-based splitting and analysis
merged_df["Date"] = pd.to_datetime(merged_df["Date"])

In [22]:
# Define a function to detect anomalies using the Z-score method
# This method calculates the Z-score for each data point within specified groups
# and flags data points with a Z-score absolute value greater than a given threshold as anomalies.
def detect_zscore(df, group_cols=["Store","Dept"], target="Weekly_Sales", z_thresh=3.0):
    out = df.copy()
    # Calculate Z-score for the target column within each group
    out["z"] = out.groupby(group_cols)[target].transform(lambda x: (x - x.mean())/x.std(ddof=0))
    # Flag data points as anomalies if the absolute Z-score exceeds the threshold
    out["anomaly_z"] = out["z"].abs() > z_thresh
    return out

In [23]:
# Apply the Z-score anomaly detection function to the merged DataFrame
# The results, including the Z-score and anomaly flag, are stored in a new DataFrame df_z.
df_z = detect_zscore(merged_df, z_thresh=3.0)

In [24]:
# Display the DataFrame with Z-score anomaly detection results
display(df_z)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Weekly_Sales_MA4,Weekly_Sales_STD4,Weekly_Sales_MA12,Weekly_Sales_STD12,IsHoliday_Lag1,IsHoliday_Lead1,Rolling_4,Rolling_12,z,anomaly_z
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.00,0.00,0.00,...,0.0000,0.000000,0.000000,0.000000,False,True,0.0000,0.000000,0.245542,False
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.00,0.00,0.00,...,0.0000,0.000000,0.000000,0.000000,False,False,0.0000,0.000000,2.395781,False
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.00,0.00,0.00,...,0.0000,0.000000,0.000000,0.000000,True,False,0.0000,0.000000,1.943233,False
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.00,0.00,0.00,...,0.0000,0.000000,0.000000,0.000000,False,False,0.0000,0.000000,-0.316684,False
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.00,0.00,0.00,...,32990.7700,12832.106391,0.000000,0.000000,False,False,32990.7700,0.000000,-0.069800,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,64.88,3.997,4556.61,20.64,1.50,...,442.9350,122.132155,565.753333,157.464751,False,False,442.9350,565.753333,-0.142924,False
421566,45,98,2012-10-05,628.10,False,64.89,3.985,5046.74,0.00,18.82,...,483.5175,104.966869,553.146667,155.304907,False,False,483.5175,553.146667,0.180750,False
421567,45,98,2012-10-12,1061.02,False,54.47,4.000,1956.28,0.00,7.89,...,552.4325,76.984436,547.554167,150.869134,False,False,552.4325,547.554167,1.351092,False
421568,45,98,2012-10-19,760.01,False,56.47,3.969,2004.02,0.00,3.18,...,666.1975,271.910501,565.530833,195.748704,False,False,666.1975,565.530833,0.537351,False


In [25]:
# Define a function to detect anomalies using the Interquartile Range (IQR) method
# This method calculates the IQR for each data point within specified groups
# and flags data points outside the range of Q1 - k*IQR and Q3 + k*IQR as anomalies.
def detect_iqr(df, group_cols=["Store","Dept"], target="Weekly_Sales", k=1.5):
    out = df.copy()
    # Define a helper function to apply IQR logic
    def _iqr_flag(x):
        # Calculate the first quartile (Q1)
        q1 = x.quantile(0.25)
        # Calculate the third quartile (Q3)
        q3 = x.quantile(0.75)
        # Calculate the Interquartile Range (IQR)
        iqr = q3 - q1
        # Define the lower bound for anomaly detection
        lower = q1 - k * iqr
        # Define the upper bound for anomaly detection
        upper = q3 + k * iqr
        # Return a boolean Series indicating whether each data point is an anomaly
        return (x < lower) | (x > upper)
    # Apply the IQR flagging logic to the target column within each group
    out["anomaly_iqr"] = out.groupby(group_cols)[target].transform(_iqr_flag)
    return out

In [26]:
# Apply the IQR anomaly detection function to the merged DataFrame
# The results, including the anomaly flag based on IQR, are stored in a new DataFrame df_iqr.
df_iqr = detect_iqr(merged_df)

In [27]:
# Display the DataFrame with IQR anomaly detection results
display(df_iqr)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Weekly_Sales_Lag3,Weekly_Sales_MA4,Weekly_Sales_STD4,Weekly_Sales_MA12,Weekly_Sales_STD12,IsHoliday_Lag1,IsHoliday_Lead1,Rolling_4,Rolling_12,anomaly_iqr
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.00,0.00,0.00,...,0.00,0.0000,0.000000,0.000000,0.000000,False,True,0.0000,0.000000,False
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.00,0.00,0.00,...,0.00,0.0000,0.000000,0.000000,0.000000,False,False,0.0000,0.000000,True
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.00,0.00,0.00,...,0.00,0.0000,0.000000,0.000000,0.000000,True,False,0.0000,0.000000,True
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.00,0.00,0.00,...,24924.50,0.0000,0.000000,0.000000,0.000000,False,False,0.0000,0.000000,False
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.00,0.00,0.00,...,46039.49,32990.7700,12832.106391,0.000000,0.000000,False,False,32990.7700,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,64.88,3.997,4556.61,20.64,1.50,...,352.44,442.9350,122.132155,565.753333,157.464751,False,False,442.9350,565.753333,False
421566,45,98,2012-10-05,628.10,False,64.89,3.985,5046.74,0.00,18.82,...,605.96,483.5175,104.966869,553.146667,155.304907,False,False,483.5175,553.146667,False
421567,45,98,2012-10-12,1061.02,False,54.47,4.000,1956.28,0.00,7.89,...,467.30,552.4325,76.984436,547.554167,150.869134,False,False,552.4325,547.554167,False
421568,45,98,2012-10-19,760.01,False,56.47,3.969,2004.02,0.00,3.18,...,508.37,666.1975,271.910501,565.530833,195.748704,False,False,666.1975,565.530833,False


In [28]:
# Define a function to detect anomalies based on the rolling residual
# This method calculates the residual between the actual value and a rolling mean (12 or 4 week)
# and flags data points where the absolute residual is greater than k times the rolling standard deviation.
def detect_rolling_residual(df, group_cols=["Store","Dept"], target="Weekly_Sales", window=12, k=3):
    out = df.copy().sort_values(group_cols + ["Date"])
    # Calculate the rolling mean of the target column, shifted by 1 to avoid data leakage
    out["rolling_mean"] = out.groupby(group_cols)[target].transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    # Calculate the rolling standard deviation of the target column, shifted by 1
    out["rolling_std"]  = out.groupby(group_cols)[target].transform(lambda x: x.shift(1).rolling(window, min_periods=1).std(ddof=0).fillna(0))
    # Calculate the residual between the actual value and the rolling mean
    out["residual"] = out[target] - out["rolling_mean"]
    # Flag data points as anomalies if the absolute residual is greater than k times the rolling standard deviation
    out["anomaly_roll"] = out["residual"].abs() > (k * out["rolling_std"])
    return out

In [29]:
# Apply the rolling residual anomaly detection function to the merged DataFrame
# using a window of 12 and a threshold multiplier k of 3.
# The results, including the rolling mean, standard deviation, residual, and anomaly flag,
# are stored in a new DataFrame df_roll.
df_roll = detect_rolling_residual(merged_df, window=12, k=3)

In [30]:
# Display the DataFrame with rolling residual anomaly detection results
display(df_roll)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Weekly_Sales_MA12,Weekly_Sales_STD12,IsHoliday_Lag1,IsHoliday_Lead1,Rolling_4,Rolling_12,rolling_mean,rolling_std,residual,anomaly_roll
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.00,0.00,0.00,...,0.000000,0.000000,False,True,0.0000,0.000000,NaN,0.000000,NaN,False
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.00,0.00,0.00,...,0.000000,0.000000,False,False,0.0000,0.000000,24924.500000,0.000000,21114.990000,True
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.00,0.00,0.00,...,0.000000,0.000000,True,False,0.0000,0.000000,35481.995000,10557.495000,6113.555000,False
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.00,0.00,0.00,...,0.000000,0.000000,False,False,0.0000,0.000000,37519.846667,9089.159071,-18116.306667,False
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.00,0.00,0.00,...,0.000000,0.000000,False,False,32990.7700,0.000000,32990.770000,11112.930118,-11162.870000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,64.88,3.997,4556.61,20.64,1.50,...,565.753333,157.464751,False,False,442.9350,565.753333,565.753333,150.761022,-57.383333,False
421566,45,98,2012-10-05,628.10,False,64.89,3.985,5046.74,0.00,18.82,...,553.146667,155.304907,False,False,483.5175,553.146667,553.146667,148.693127,74.953333,False
421567,45,98,2012-10-12,1061.02,False,54.47,4.000,1956.28,0.00,7.89,...,547.554167,150.869134,False,False,552.4325,547.554167,547.554167,144.446199,513.465833,True
421568,45,98,2012-10-19,760.01,False,56.47,3.969,2004.02,0.00,3.18,...,565.530833,195.748704,False,False,666.1975,565.530833,565.530833,187.415116,194.479167,False


**B. Anomaly Detection — ML methods (IsolationForest + Autoencoder)**
**Isolation Forest (unsupervised)**

In [13]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# choose features for anomaly detection (numeric + lags)
feature_cols = ["Weekly_Sales","Weekly_Sales_Lag1","Weekly_Sales_Lag2","Weekly_Sales_MA4",
                "Temperature","Fuel_Price","CPI","Unemployment","Size"]  # adapt to your merged_df

df_ml = merged_df.dropna(subset=feature_cols).copy()
X = df_ml[feature_cols].values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

In [14]:

iso = IsolationForest(n_estimators=200, contamination=0.01, random_state=42)
iso.fit(Xs)
df_ml["iso_score"] = iso.decision_function(Xs)   # higher = more normal
df_ml["anomaly_iso"] = iso.predict(Xs) == -1     # True => anomaly


**Autoencoder (reconstruction error)**

In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Build a small autoencoder model for anomaly detection
# The autoencoder will learn to reconstruct the input data, and data points with high reconstruction error are considered anomalies.
input_dim = Xs.shape[1]  # Number of features in the input data
encoding_dim = max(8, input_dim//2) # Dimension of the encoded representation, at least 8 or half of input dimension

# Define the input layer
inp = layers.Input(shape=(input_dim,))
# Define the encoder layers with ReLU activation
encoded = layers.Dense(encoding_dim, activation="relu")(inp)
encoded = layers.Dense(encoding_dim//2, activation="relu")(encoded)
# Define the decoder layers with ReLU activation, reconstructing the original dimension
decoded = layers.Dense(encoding_dim, activation="relu")(encoded)
decoded = layers.Dense(input_dim, activation="linear")(decoded) # Linear activation for output layer

# Create the autoencoder model
autoencoder = models.Model(inp, decoded)
# Compile the autoencoder with Adam optimizer and Mean Squared Error loss
autoencoder.compile(optimizer="adam", loss="mse")

# Train the autoencoder on the scaled data
# Train for 30 epochs with a batch size of 256, using 10% of data for validation.
# verbose=2 shows training progress per epoch.
autoencoder.fit(Xs, Xs, epochs=30, batch_size=256, validation_split=0.1, verbose=2)

# Calculate the reconstruction error (Mean Squared Error) for each data point
recon = autoencoder.predict(Xs)
mse = np.mean(np.square(Xs - recon), axis=1)
# Add the reconstruction error to the DataFrame
df_ml["ae_mse"] = mse

# Determine the threshold for anomaly detection based on reconstruction error
# Here, the top 1% of data points with the highest MSE are considered anomalies.
threshold = np.percentile(mse, 99)
# Flag data points as anomalies if their MSE is above the calculated threshold
df_ml["anomaly_ae"] = df_ml["ae_mse"] > threshold

Epoch 1/30
1483/1483 - 8s - 5ms/step - loss: 0.3669 - val_loss: 0.1809
Epoch 2/30
1483/1483 - 7s - 5ms/step - loss: 0.1449 - val_loss: 0.1644
Epoch 3/30
1483/1483 - 6s - 4ms/step - loss: 0.1210 - val_loss: 0.1641
Epoch 4/30
1483/1483 - 4s - 3ms/step - loss: 0.1191 - val_loss: 0.1659
Epoch 5/30
1483/1483 - 3s - 2ms/step - loss: 0.1187 - val_loss: 0.1653
Epoch 6/30
1483/1483 - 5s - 4ms/step - loss: 0.1182 - val_loss: 0.1639
Epoch 7/30
1483/1483 - 5s - 3ms/step - loss: 0.1176 - val_loss: 0.1623
Epoch 8/30
1483/1483 - 4s - 2ms/step - loss: 0.1166 - val_loss: 0.1604
Epoch 9/30
1483/1483 - 5s - 3ms/step - loss: 0.1156 - val_loss: 0.1594
Epoch 10/30
1483/1483 - 5s - 3ms/step - loss: 0.1147 - val_loss: 0.1588
Epoch 11/30
1483/1483 - 4s - 3ms/step - loss: 0.1141 - val_loss: 0.1553
Epoch 12/30
1483/1483 - 4s - 2ms/step - loss: 0.1136 - val_loss: 0.1580
Epoch 13/30
1483/1483 - 5s - 3ms/step - loss: 0.1132 - val_loss: 0.1573
Epoch 14/30
1483/1483 - 5s - 4ms/step - loss: 0.1129 - val_loss: 0.1548
E

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

# Define a function to calculate classification metrics (accuracy, precision, recall, F1-score)
# This function is used to evaluate the performance of anomaly detection models
# against a set of true labels (or proxy labels in this case).
def classification_metrics(y_true, y_pred):
    # Calculate accuracy
    acc = accuracy_score(y_true, y_pred)
    # Calculate precision
    prec = precision_score(y_true, y_pred)
    # Calculate recall
    rec = recall_score(y_true, y_pred)
    # Calculate F1-score
    f1 = f1_score(y_true, y_pred)
    # Return a dictionary containing all calculated metrics
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

In [33]:
# Create a proxy label for anomalies based on the rolling residual method
# Data points with absolute residuals in the top 1% are considered proxy anomalies.
proxy = df_roll.copy()
proxy["proxy_anomaly"] = proxy["residual"].abs() > proxy["residual"].abs().quantile(0.99)

# Merge the proxy anomaly labels with the DataFrame used for ML anomaly detection (df_ml)
# This aligns the proxy labels with the results from Isolation Forest and Autoencoder.
df_eval = df_ml.merge(proxy[["Store","Dept","Date","proxy_anomaly"]], on=["Store","Dept","Date"], how="left")
# Fill any missing proxy anomaly values (due to the merge) with False
df_eval["proxy_anomaly"] = df_eval["proxy_anomaly"].fillna(False)

# Evaluate the performance of Isolation Forest using classification metrics against the proxy anomalies
print("IsolationForest vs proxy:", classification_metrics(df_eval["proxy_anomaly"], df_eval["anomaly_iso"]))
# Evaluate the performance of the Autoencoder using classification metrics against the proxy anomalies
print("Autoencoder vs proxy:", classification_metrics(df_eval["proxy_anomaly"], df_eval["anomaly_ae"]))

IsolationForest vs proxy: {'accuracy': 0.9825295917641198, 'precision': 0.1226280834914611, 'recall': 0.12359550561797752, 'f1': 0.12310989403500416}
Autoencoder vs proxy: {'accuracy': 0.9841283772564462, 'precision': 0.20256166982922202, 'recall': 0.20415969399952189, 'f1': 0.20335754256459102}


In [34]:
# This is an empty cell and can be used for further analysis or code.